In [7]:
import json
import plotly.graph_objects as go
import numpy as np
import open3d as o3d

from pathlib import Path

from utils.misc import get_instruction

SPLIT = 'dev'
DATASET_ROOT = Path('/home/miks/dataset')

subs = Path('/home/miks/submission_opensun3d_track2')
masks_pth = subs / 'predicted_masks'
ins_pth = Path('/home/miks/letsdoit/sample_instructions.json')

with open(ins_pth, 'rb') as f:
    instructions = json.load(f)

fig = None

def visualize_pointcloud_o3d(pcd, obj, instruction_str):
    # Create Open3D PointCloud objects
    
    obj_pcd = o3d.geometry.PointCloud()
    obj_pcd.points = o3d.utility.Vector3dVector(obj)
    
    # Create colors for obj points (red)
    obj_colors = np.array([[1, 0, 0] for _ in range(len(obj))])
    obj_pcd.colors = o3d.utility.Vector3dVector(obj_colors)
    
    # Visualize
    o3d.visualization.draw_geometries([pcd, obj_pcd])

for mask_pth in masks_pth.glob('*.txt'):
    visit_id, desc_id, _ = mask_pth.name.split('_')
    instruction_str = get_instruction(instructions, visit_idd, desc_id)
    print(instruction_str)
    pcd_path = DATASET_ROOT / SPLIT / visit_id / f'{visit_id}_laser_scan.ply'
    crop_mask_path = DATASET_ROOT / SPLIT / visit_id / f'{visit_id}_crop_mask.npy'

    # Read point cloud
    pcd = o3d.io.read_point_cloud(str(pcd_path))
    points = np.asarray(pcd.points)
    mask_idx = np.loadtxt(mask_pth, dtype=bool)
    crop_mask = np.load(crop_mask_path)

    obj = points[mask_idx, :]
    pcd_cropped = pcd.select_by_index(np.where(crop_mask)[0])
    visualize_pointcloud_o3d(pcd_cropped, obj, instruction_str)
